# sample generator for who, where, how
This notebook is for creating the random sample .txt files for the experiment.

In [1]:
import pandas as pd
import numpy as np

 ## Contents
 1. [Create separate file with contexts](#Create-separate-file-with-contexts)
 2. [Create separate file without contexts](#Create-separate-file-without-contexts)
 3. [Creating the files for the experiment](#Creating-files-for-the-experiment)
 4. [Automate Paraphrase Generator](#Automate-pharaphrase-generator)


In [2]:
# import the database file from the TGrep2 searching
df = pd.read_csv("../results/swbd.tab", sep='\t', engine='python')

In [3]:
# This makes the display show more info
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Creating the files for the experiment

## Constrain dataset
for experimental mock-up

First we have to remove the questions that we don;t want to include:
1. non-embedded or root questions
2. no degree questions
3. no identity questions
4. generally only monomorphemic wh-phrases
5. only who-, where-, and how-questions

In [7]:
# distribution before 

In [9]:
df.columns

Index(['Item_ID', 'Sentence', 'HaveNeedTo', 'Finite', 'ModalPresent',
       'QuestionType', 'DegreeQ', 'SubjectAuxInv', 'WhAll', 'MatrixNegPresent',
       'EmbeddedNegPresent', 'QuantifiedSubject', 'QuantifiedPredicate', 'Wh',
       'MatrixNegation', 'MatrixDoSupport', 'MatrixPred1', 'MatrixPred2',
       'MatVerbPart', 'Modal', 'EmbeddedNegation', 'Verb1', 'Verb2', 'Verb3',
       'DeterminerSubject', 'DeterminerNonSubject', 'FullWhPhrase',
       'DeterminerSubjPresent', 'DeterminerNonSubjPresent', 'WhNode',
       'WhParse', 'Question', 'SentenceParse', 'WhPhaseType', 'IdentityQ'],
      dtype='object')

In [11]:
df.pivot_table(index=['QuestionType'], values="Question", aggfunc=len).groupby(["QuestionType"]).Question.transform(lambda x: x/len(df)).reset_index()

,QuestionType,Question
0,adjunct,0.075988
1,embadjunct,0.235415
2,embedded,0.295911
3,exclamation,0.003334
4,fragment,0.012354
5,relative,0.135111
6,root,0.168546
7,subject,0.073341


In [ ]:
df.pivot_table(index=['QuestionType'], values="Question", aggfunc=len).groupby(["QuestionType"]).Question.transform(lambda x: x/len(df)).reset_index()

In [3]:
critical = df[(df['QuestionType'] == 'root') # only root questions
              & 
              (df['DegreeQ'] == 'no' ) # no degree questions
              &
              (df['IdentityQ'] == "no") # no identity questions
              &
              (df['Wh'].isin(['how','How','where','Where','who','Who'])) # just these wh-words
              &
              (df['WhPhaseType'] == 'monomorphemic') # monomorphic wh only (might get anything not go by degQ)
             ]

In [12]:
critical.pivot_table(index=['ModalPresent'], values="Question", aggfunc=len).groupby(["ModalPresent"]).Question.transform(lambda x: x/len(critical)).reset_index()

,ModalPresent,Question
0,no,0.880597
1,yes,0.119403


In [14]:
critical.pivot_table(index=['Wh'], values="Question", aggfunc=len).groupby(["Wh"]).Question.transform(lambda x: x/len(critical)).reset_index()

,Wh,Question
0,how,0.549254
1,where,0.289552
2,who,0.161194


In [18]:
critical.pivot_table(index=['Wh','ModalPresent'], values="Question", aggfunc=len).groupby(["Wh"]).Question.transform(lambda x: x/len(critical)*100).reset_index()

,Wh,ModalPresent,Question
0,how,no,46.567164
1,how,yes,8.358209
2,where,no,27.761194
3,where,yes,1.194030
4,who,no,13.731343
5,who,yes,2.388060


should contain about 4 modal in each batch
10 lists of 30
1 list of 35

balance so that there are 3 modals 


QUESTION PHRASING
avoid using 'paraphrase'
1. based on the question in red, which of the following does the speaker want to know?
2. "none of the above"
3. ellipsis before the 
4. practice examples that are fake from the literature clear MS/MA answers....
4. get rid of trace

should be a way to highlight just the question, and not the entire sentence
"how can you tell where it's going to come down at"


is there a way to reorder the elements of the question to create the paraphrase
strip the "do" and insert "that"


rhetorical questions: "so my family may say that on occasion, who knows"

Piloting
10 items that will be repiloted for each different testing 

vary between subjects the order of the paraphrases
sample randomly at the beginning of the experiment what the order is
but keep it consistent in the experiment and change between subjects


By monday post on proliferate*



look at the lab wifi for pre-registration on OSF

get R going....

## Paraphrase Generator
this should take as input the entire constrained dataframe from the above section, and then generate the paraphrases

For Who questions: Who is a person...? / Who is some person...? / "Who is every person..." / "Who is the person..."

In [4]:
# read in df with contexts
cntxts = pd.read_csv("swbd_contexts.csv")

In [5]:
cntxts = cntxts.drop(columns="FollowingContext")

In [6]:
# get the indixes from critical
crit_index = critical.Item_ID

Merge back in Wh and ModalPresent colums

In [7]:
df_WhMod = critical[["Item_ID","Wh","ModalPresent"]].rename(columns={"Item_ID": "TGrepID"})

In [8]:
# subset to the items that are just the ones filtered in the previos section

# otherwise, if using the database file with contexts directly in there, then this step
# is not necessary
df_valid = cntxts[cntxts["TGrepID"].isin(set(crit_index))]

In [10]:
len(df_valid)

335

In [9]:
df_valid = df_valid.merge(df_WhMod, how = 'inner', indicator=False)

In [11]:
len(df_valid)

335

In [268]:
df_valid.pivot_table(index=['Wh'], values="EntireSentence", aggfunc=len).groupby(["Wh"]).EntireSentence.transform(lambda x: x/len(df_valid)).reset_index()

,Wh,EntireSentence
0,how,0.549254
1,where,0.289552
2,who,0.161194


In [10]:
who = df_valid[df_valid["Wh"] == "who"]
where = df_valid[df_valid["Wh"] == "where"]
how = df_valid[df_valid["Wh"] == "how"]

In [11]:
who["AResponse"] = "Who is a person...?"
# who["SomeResponse"] = "Who is some person...?"
who["AllResponse"] = "Who is every person...?"
who["TheResponse"] = "Who is the person...?"


where["AResponse"] = "What is a place...?"
# where["SomeResponse"] = "What is some place...?"
where["AllResponse"] = "What is every place...?"
where["TheResponse"] = "What is the place...?"


how["AResponse"] = "What is a way...?"
# how["SomeResponse"] = "What is some way...?"
how["AllResponse"] = "What is every way...?"
how["TheResponse"] = "What is the way...?"

<ipython-input-11-db43c0f19116>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  who["AResponse"] = "Who is a person...?"
<ipython-input-11-db43c0f19116>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  who["AllResponse"] = "Who is every person...?"
<ipython-input-11-db43c0f19116>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [12]:
df_final = pd.concat([who,where,how])

In [15]:
len(df_final)

335

In [16]:
df_final.pivot_table(index=['Wh'], values="EntireSentence", aggfunc=len).groupby(["Wh"]).EntireSentence.transform(lambda x: x/len(df_final)).reset_index()

,Wh,EntireSentence
0,how,0.549254
1,where,0.289552
2,who,0.161194


In [17]:
df_final.pivot_table(index=['ModalPresent'], values="EntireSentence", aggfunc=len).groupby(["ModalPresent"]).EntireSentence.transform(lambda x: x/len(df_final)).reset_index()

,ModalPresent,EntireSentence
0,no,0.880597
1,yes,0.119403


There should be a way to automate that whole thing in one definitieion, but im failing to do that correctly

## Create randomly sampled files

### TEST SAMPLE

deseridata:
15 how , 10 where, 5 who

This is possible to maintain in lists 1-9, but lists 10-11 have slightly different distributions.

should contain about 3 modal in each batch, lists 10-11 have 6/7 respectively

10 lists of 30
1 list of 35





In [16]:
controls = pd.read_csv("../../experiments/clean_corpus/controls.csv")

In [17]:
controls["Wh"] = "none"
controls["ModalPresent"] = "no"

In [18]:
controls = controls[["TGrepID","EntireSentence","PreceedingContext","Wh","ModalPresent","AResponse","AllResponse","TheResponse"]]

In [19]:
controls

,TGrepID,EntireSentence,PreceedingContext,Wh,ModalPresent,AResponse,AllResponse,TheResponse
0,control_a_tissue,Do you want a tissue?,speakera. Your nose is running. speakerb. Oh y...,none,no,A tissue…?,Every tissue…?,The tissue…?
1,control_a_napkin,Can I hand you a napkin?,speakera. Oh no! I just spilled my spaghetti d...,none,no,A napkin…?,Every napkin…?,The napkin...?
2,control_the_book,Did you return the book you borrowed?,speakera. Hey. speakerb. Hey what's up? speake...,none,no,A book...?,Every book…?,The book...?
3,control_the_movie,Can you text me the movie schedule?,speakera. What are you doing tonight? speakerb...,none,no,A movie schedule...?,Every movie schedule…?,The movie schedule...?
4,control_all_cookies,Did you grab all the cookies?,speakera. It's time to leave for the potluck.T...,none,no,A cookie...?,Every cookie…?,The cookie...?
5,control_all_novels,Have you read all her novels?,speakera. I love Octavia Butler she's the best...,none,no,An novel...?,Every novel…?,The novel...?


In [22]:
df_final.groupby(["ModalPresent"])["ModalPresent"].count()

ModalPresent
no     295
yes     40
Name: ModalPresent, dtype: int64

In [23]:
40 - 11*3

7

In [24]:
df_final.groupby(["Wh"])["Wh"].count()

Wh
how      184
where     97
who       54
Name: Wh, dtype: int64

In [465]:
54/10

5.4

In [15]:
df_final = pd.concat([who,where,how])

In [491]:
print(df_final.groupby(["ModalPresent"])["ModalPresent"].count())

ModalPresent
no     295
yes     40
Name: ModalPresent, dtype: int64


In [ ]:
# print(df_final.groupby(["ModalPresent"])["ModalPresent"].count())

In [19]:
len(test)

335

In [14]:
len(df_final)

305

In [20]:
# ids = pd.DataFrame(columns=df_final.columns)
for n in range(1,10):
#     print(df_final.groupby(["ModalPresent"])["ModalPresent"].count())
    mod_sample = df_final[df_final["ModalPresent"] == "yes"].sample(3)

    i = len(mod_sample[mod_sample["Wh"] == "how"])
    j = len(mod_sample[mod_sample["Wh"] == "who"])
    k = len(mod_sample[mod_sample["Wh"] == "where"])
    df_final = df_final.drop(mod_sample.index)

    how_sample = df_final[
        (df_final["Wh"] == "how") &
        (df_final["ModalPresent"] == "no")
                         ].sample(15-i)
    df_final = df_final.drop(how_sample.index)

    who_sample = df_final[
        (df_final["Wh"] == "who") &
        (df_final["ModalPresent"] == "no")].sample(5-j)
    df_final = df_final.drop(who_sample.index)

    where_sample = df_final[
        (df_final["Wh"] == "where") &
        (df_final["ModalPresent"] == "no")].sample(10-k)
    df_final = df_final.drop(where_sample.index)

    total = pd.concat([mod_sample,how_sample,who_sample,where_sample,controls])
    
    # save to file
    filename = f"../../experiments/clean_corpus/corpus_{n}.txt".format(n=n)
    total.to_csv(filename,header=True,sep="\t",index=False)

In [26]:
len(df_final)

65

In [27]:
df_final.groupby(["Wh"])["Wh"].count()

Wh
how      49
where     7
who       9
Name: Wh, dtype: int64

who
9/2 = 4.5

who
9/2 = 3.5

how
49/2 = 24.5

In [533]:
df_final.groupby(["ModalPresent"])["ModalPresent"].count()

ModalPresent
no     52
yes    13
Name: ModalPresent, dtype: int64

In [506]:
13/2

6.5

Second iteration

In [28]:
for n in range(10,11):
    mod_sample = df_final[df_final["ModalPresent"] == "yes"].sample(6)
#     print(len(mod_sample))
    i = len(mod_sample[mod_sample["Wh"] == "how"])
    j = len(mod_sample[mod_sample["Wh"] == "who"])
    k = len(mod_sample[mod_sample["Wh"] == "where"])
    df_final = df_final.drop(mod_sample.index)
#     print(len(df_final))

    how_sample = df_final[
        (df_final["Wh"] == "how") &
        (df_final["ModalPresent"] == "no")
                         ].sample(23-i)
    df_final = df_final.drop(how_sample.index)
    print(len(df_final))

    who_sample = df_final[
        (df_final["Wh"] == "who") &
        (df_final["ModalPresent"] == "no")].sample(4-j)
    df_final = df_final.drop(who_sample.index)
    print(len(df_final))

    where_sample = df_final[
        (df_final["Wh"] == "where") &
        (df_final["ModalPresent"] == "no")].sample(3-k)
    df_final = df_final.drop(where_sample.index)

    total = pd.concat([mod_sample,how_sample,who_sample,where_sample,controls])
    print(len(total))
#     save to file
    filename = f"../../experiments/clean_corpus/corpus_{n}.txt".format(n=n)
    total.to_csv(filename,header=True,sep="\t",index=False)

39
37
36


In [29]:
23+4+3

30

In [30]:
len(df_final)

35

In [31]:
df_final.groupby(["ModalPresent"])["ModalPresent"].count()

ModalPresent
no     28
yes     7
Name: ModalPresent, dtype: int64

In [32]:
df_final.groupby(["Wh"])["Wh"].count()

Wh
how      26
where     4
who       5
Name: Wh, dtype: int64

In [33]:
last = pd.concat([df_final,controls])

In [34]:
last.to_csv("../../experiments/clean_corpus/corpus_11.txt",header=True,sep="\t",index=False)

## Pilot Samples

### Pilot Ex 01

In [98]:
# since df_final changed, it's a different sample set now.
pilot = df_final.sample(10, random_state=333)

In [22]:
# pilot.to_csv("../../experiments/clean_corpus/pilot1.txt",header=True,sep="\t",index=False)

### Pilot Ex 02
This design uses sliders rather than radio buttons, and is not a forced choice.

Stimuli are the same as Pilot Ex01, but with rhetorical questions replaced.

In [23]:
# rhetorical questions to remove
# 5928, 2328
pilot.EntireSentence

3582                                                    how are your kids handling it *t*-1?
3511                                                                        who *t*-1 knows.
2178                           well how do you think 0 the royals are going *-1 to do *t*-2?
5928    but, uh, who *t*-1 knows what the future will hold *t*-2. uh, as far as energy goes,
5859                                      how do you usually find a, a new restaurant *t*-1?
2931                                                           uh, who's the last one *t*-1.
7463                          oh, g-, where *ich*-3 have i gone *t*-1 that *t*-2's anywhere.
2328        you know, how can you help *-1 depending on the day care center *t*-2, you know,
6779                                                  uh, how do you feel about taxes *t*-1?
3938                                                                  where do you go *t*-1?
Name: EntireSentence, dtype: object

In [24]:
sample2 = df_final.sample(2, random_state=33)

In [25]:
sample2.PreceedingContext

5347    ###that's all mexico, i think 0 *t*-1.###speakerb57.###yes,###uh-huh.### and, but it was wonderful.###so we really like those.###and, uh, i guess 0 we just like *-1 to go someplace, someplace entirely different.###we've been to a few places in this country.###we still have a lot 0 *-1 to see *t*-2, though.###speakera58.
7270                                                                                                              ###i mean, pretty much it's a white shirt and slacks and shoes and a coat.###speakerb12.###uh-huh.###speakera13.###speakerb14.###uh-huh.###speakera15.###speakerb16.###speakera17.###but, it was very different before.
Name: PreceedingContext, dtype: object

In [26]:
sample2.EntireSentence

5347    where are you from *t*-1 originally?
7270                 how do you dress *t*-1?
Name: EntireSentence, dtype: object

In [27]:
pilot = pilot.drop([5928,2328], axis=0)

In [32]:
pilot2 = pd.concat([pilot, sample2])

In [36]:
# remove the someresponse column
pilot2 = pilot2.drop(columns="SomeResponse")

In [37]:
pilot2.head(1)

,TGrepID,EntireSentence,PreceedingContext,AResponse,AllResponse,TheResponse
3582,61578:4,how are your kids handling it *t*-1?,"###speakera161.###no,### i, i know,###i mean he's always, you know, he really does have a good home.###i think 0 that's what *t*-1 really bothers me the most about it.###speakerb162.###yeah.###speakera163.###you know, it's like,###speakerb164.",What is a way...?,What is every way...?,What is the way...?


In [38]:
pilot2.to_csv("../../experiments/clean_corpus/pilot2.txt",header=True,sep="\t",index=False)

## Pilot 03

first pilot set with controls

In [35]:
pilot1 = pd.read_csv("../../experiments/clean_corpus/pilot1.txt", sep="\t", engine='python')

In [36]:
pilot1 = pilot1.drop(columns=["SomeResponse"])

In [37]:
cont = controls.drop(columns=["Wh","ModalPresent"])

In [39]:
pilot3 = pd.concat([pilot1,cont])

In [40]:
pilot3.to_csv("../../experiments/clean_corpus/pilot3.txt",header=True,sep="\t",index=False)

In [41]:
pilot3

,TGrepID,EntireSentence,PreceedingContext,AResponse,AllResponse,TheResponse
0,61578:4,how are your kids handling it *t*-1?,"###speakera161.###no,### i, i know,###i mean he's always, you know, he really does have a good home.###i think 0 that's what *t*-1 really bothers me the most about it.###speakerb162.###yeah.###speakera163.###you know, it's like,###speakerb164.",What is a way...?,What is every way...?,What is the way...?
1,60430:4,who *t*-1 knows.,"### and, and i'm getting rather good at *-1 reading want ads.###speakerb14.###uh-huh.###speakera15.###yeah,### it, it,###speakerb16.### wh-, what kind of time frame is it, uh, looking *t*-1 for the, uh, different shifts up there?###speakera17.###i,",Who is a person...?,Who is every person...?,Who is the person...?
2,37508:7,well how do you think 0 the royals are going *-1 to do *t*-2?,"###they said the, that um, that he was horrible at, uh, at *-1 batting with, with men on base.###now * having somebody get up and hit a solo home run does you, you know, gives you run, obviously.###speakerb34.###yes.###speakera35.###but i mean if you can't do it with men on base, there's something definitely wrong, um, you know,### you, you've got *-1 to be able *-2 to get the two and three run home runs, you know, *-2 to, to be able *-3 to, uh, score enough runs * to be able *-4 to win in these, uh, in this day and age,###so.###speakerb36.###yeah,",What is a way...?,What is every way...?,What is the way...?
3,103191:13,"but, uh, who *t*-1 knows what the future will hold *t*-2. uh, as far as energy goes,","###speakera61.###yes.###speakerb62.###it's just 0 we can't control it.###right now it *exp*-1 takes more energy *ich*-2 * to cause fusion to occur then you get *?* out,###so, it isn't *?*,###not at all,###and you can't control it unless you gotten a huge electromagnetic field,###so that's not going *-1 to work.###at least not now.",Who is a person...?,Who is every person...?,Who is the person...?
4,102105:4,"how do you usually find a, a new restaurant *t*-1?","###no,###i don't think so.###speakerb52.###but, we really, we really do enjoy *-1 going out### and uh, and, um, i'm pretty adventurous in cooking, too, which *t*-1 also occasionally backfires.###speakera53.###oh, well. even though traditional it can backfire.###speakerb54.###yeah.###i think 0 we're supposed *-1 to say what *t*-2 will bring us back to a restaurant or, or what *t*-3 attracts us to a restaurant.",What is a way...?,What is every way...?,What is the way...?
5,51293:9,"uh, who's the last one *t*-1.","###right.###speakera46.###speakerb47.###well, * picking up darrell strawberry didn't hurt.###speakera48.###no,###but butler didn't hurt either.###uh, and bobby ojeda didn't hurt###and,###speakerb49.",Who is a person...?,Who is every person...?,Who is the person...?
6,130644:22,"oh, g-, where *ich*-3 have i gone *t*-1 that *t*-2's anywhere.","###speakerb70.###okay.###speakera71.###speakerb72.###well, uh, well,###speakera73.###i'm wanting *-1 to go there.###speakerb74.###where have you gone *t*-1 first?###speakera75.",What is a place...?,What is every place...?,What is the place...?
7,40349:14,"you know, how can you help *-1 depending on the day care center *t*-2, you know,","###they can watch,###and i just kind of always felt that the chances of something happening were less.###speakera35.###yeah,###yeah.###speakerb36.###speakera37.###uh-huh.###because there's,###speakerb38.",What is a way...?,What is every way...?,What is the way...?
8,118088:9,"uh, how do you feel about taxes *t*-1?","###speakera3.###oh, right###indianapolis got that, didn't they?###speakerb4.###right.###and, uh, we had passed a, a provisional one that if, if it did, if they did locate here we would raise our taxes *t*-1.###but, uh, at least we thought 0 we'd get something back from that.###speakera5.###yeah.###speakerb6.",What is a way...?,What is every way...?,What is the way...?
9,68569:4,where do you go *t*-1?,"###and we go to florida###speakera21.###uh.###spea